In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns
import math

from keras.models import Sequential
from keras.layers import Dense, LSTM, SimpleRNN, Activation
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from keras.callbacks import ModelCheckpoint, EarlyStopping

%matplotlib inline

C:\Users\Gj\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


# Loading Dataset

In [2]:
stock_exchange_mk = pd.read_csv('data_mse_mse_historic_data_active.csv', sep='\t', parse_dates=['Date'])

In [3]:
df = pd.DataFrame(stock_exchange_mk)
df["stock_id"] = df["stock_id"].astype('category')
df.columns = df.columns.str.lower()

# Missing Dates

In [103]:
stocks = df.stock_id.unique()
dates_all = df.set_index('date').index
idx = pd.MultiIndex.from_product((dates_all, stocks), names=['date', 'stock_id'])
df = df.set_index(['date', 'stock_id']).reindex(idx, fill_value=0).reset_index()

In [169]:
date_index = []
for i in df_list:
    idx = min(i.date)[0][1]
    date_index.append(idx)

In [176]:
start_date = dict(zip(stocks, date_index))

In [192]:
start_date_df = pd.DataFrame.from_dict(start_date, orient="index")
start_date_df

,0
ALK,1997-01-09
BESK,2002-10-10
FERS,1999-04-15
GRNT,1999-10-28
KMB,1997-05-15
MPT,1998-05-19
MTUR,1998-07-23
OKTA,2000-03-21
PKB,1999-11-09
PPIV,1999-05-18


In [191]:
for i in df.drop()

,stock_id,date
0,ALK,2020-08-25
1,BESK,2020-08-25
2,FERS,2020-08-25
3,GRNT,2020-08-25
4,KMB,2020-08-25
...,...,...
955697,TNB,1998-10-13
955698,TTK,1998-10-13
955699,UNI,1998-10-13
955700,USJE,1998-10-13
